In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

Cargar los datos

In [ ]:
df = pd.read_csv('data.csv', sep=';')
df

Encontrar los valores basicos de cada columna numerica, como la media, la mediana, la desviacion estandar, el minimo y el maximo

In [ ]:
df.describe()

In [ ]:
print(df.info())

Se puede ver que no existen valores nulos en los datos, entonces no se tiene necesidad de hacer una limpieza de nulos

In [ ]:
df.head()

Se puede notar que hay varias variables categoricas en el dataset

Vamos a conseguir nuestras variables categoricas y numericas, para graficarlas y ver como se comportan

In [ ]:
numeric = []
for col in df.columns:
    if df[col].dtype != 'O':
        numeric.append(col)

print(f"Varaibles numericas: {numeric}")

categorical = []
for col in df.columns:
    if df[col].dtype == 'O':
        categorical.append(col)

print(f"Variables categoricas: {categorical}")

In [ ]:
for num in numeric:
    plt.boxplot(df[num])
    plt.title(num)
    plt.show()

Vamos a ver cuantos outliers tenemos en cada una de nuestras columnas numericas

In [ ]:
for num in numeric:
    iqr = df[num].quantile(0.75) - df[num].quantile(0.25)
    print(f"Numero outliers de {num} es: {(df[num] > df[num].quantile(0.75) + 1.5 * iqr).sum() + (df[num] < df[num].quantile(0.25) - 1.5 * iqr).sum()}")

Podemos notar que tenemos muchos outliers en nuestras columnas, aunque con respecto a el tamaño de nuestro dataset, no es un numero tan grande.
El problema es que en columnas como la edad, si que nos interesa guardar los outliers, puesto que existe la posibilidad de que la gente de mayor edad tienda a nunca abrir un CDT, o la gente con mucho dinero en la cuenta si que tienda a abrir un CDT.
<br>
Aun asi notemos que en nuestra columna de `previous` si que hay un valor ya muy extremo, ese si que va a ser eliminado 100%

De igual manera, vamos a calcular el numero de valores extremos (3IQR)

In [ ]:
for num in numeric:
    iqr = df[num].quantile(0.75) - df[num].quantile(0.25)
    print(f"Numero valores extremos de {num} es: {(df[num] > df[num].quantile(0.75) + 3 * iqr).sum() + (df[num] < df[num].quantile(0.25) - 3 * iqr).sum()}")

Algo que podemos notar sabiendo el contexto de nuestro dataset, es que la columna `previous`  y  `pdays` son columnas en las que tienen muchos extremos debido a que la gran mayoria de las personas contactadas, son contactadas por primera vez, por ende el valor de `pdays` es -1 y `previous` es 0.
<br>
Esto tambien se puede comprobar revisando los cuartiles de estas variables.

Adicionalmente, vamos a mirar que distribucion tienen nuestras variables numericas

In [ ]:
for num in numeric:
    plt.hist(df[num], bins = 40)
    plt.axvline(x=df[num].mean(), color='r', linestyle='dashed', linewidth=2)  # La media
    plt.title(num)
    plt.show()

A pesar de que la columna previous no se puede ver muy bien en el grafico, no importa ya que por la naturaleza del problema escogido, esta columna se va a desechar de igual manera

### variables categoricas

In [ ]:
for cat in categorical:
    df[cat].value_counts().plot(kind='bar')
    plt.title(cat)
    plt.show()

Podemos notar cosas como que:
- a simple vista se puede ver que los trabajadores manuales, los gerente y los tecnicos representan poco mas de la mitad de todos los contactados.
- Mas de la mitad de los contactados son casados.
- Mas o menos la mitad solo llegaron al bachillerato
- Casi ningun contactado ha incumplido en la deuda
- La mayoria de los contactados no tienen ningun prestamo
- Mucho mas de la mitad de los contactados fueron por medio del celular
- En mayo se hacen mas o menos el doble de llamadas (o contactos pues) que en el siguiente mes
- casi no se sabe ningun resultado de las campañas anteriores de marketing
- Solo el 12% (medido a ojo) de los contactos resultan en que los clientes abran un CDT en el banco

### Relaciones entre variables con nuestro target

In [ ]:
for num in numeric:
    sn.boxplot(x='y', y=num, data=df)
    plt.show()

In [ ]:
for num in numeric:
    sn.histplot(df, x=num, hue='y', multiple='stack')
    plt.show()

Algo notable es que las personas jovenes y las personas mayores son en porcentaje de contactadas, las que mas tienden a abrir un CDT.

Ahora las variables categoricas

In [ ]:
for cat in categorical:
    if cat == 'y': continue

    agg_df = df.groupby(cat).agg(
        total=('y', 'count'),
        p_yes=('y', lambda x: (x=='yes').sum()),
        p_no=('y', lambda x: (x=='no').sum())
    )

    agg_df['p_yes'] = agg_df['p_yes'] * 100 / agg_df['total']
    agg_df['p_no'] = agg_df['p_no'] * 100 / agg_df['total']

    plot = agg_df[['p_yes', 'p_no']].plot(kind='bar', stacked=True)
    plt.xticks(rotation='vertical')

    for i, val in enumerate(agg_df['p_yes'].values):
        plt.text(i, val, f"{round(agg_df['p_yes'].values[i], 1)}%", ha='center', va='bottom')

    plot.legend(['Yes', 'No'])
    plt.show()

Podemos notar cosas interesantes, como :
- los clientes que han accedido a un producto del banco anteriormente, son muchisimo mas propensos a abrir un CDT. 
- las personas que no tienen prestamos de bienes raices son mas del doble de probables de abrir un cdt que las personas que cuentan con estos prestamos.
- el modo de contacto no aparenta ser un factor a tener en cuenta, esto debido a que las probabilidades de abrir un CDT son muy similares en ambos modos de contacto